# Libraries

In [ ]:
import yaml
from pathlib import Path

from src.utils import  get_data_info, train_val_test_split
from src.classes import NeuronPatchDataset

import tifffile
import os

# Helper Functions

# Main

### read config

In [ ]:
config_yaml_path = "config.yaml"

config_path = Path(config_yaml_path)

with config_path.open("r", encoding="utf-8") as f:
    cfg = yaml.safe_load(f) 

data_path = cfg["dir"]["input"]
output_dir = cfg["dir"]["output"]

tiff_suffixes = cfg["input_data"]["tiff_suffixes"]
marker_suffix = cfg["input_data"]["marker_suffix"]

ground_truth_config = cfg["ground_truth_config"]

pre_processing_config = cfg["pre_processing_config"]

print(data_path)
print(output_dir)

print(tiff_suffixes)
print(marker_suffix)

print(ground_truth_config)
print(pre_processing_config)

### create dataframe with all the data info

In [ ]:
df = get_data_info(
    data_path=data_path,
    tiff_suffixes=tiff_suffixes,
    marker_suffix=marker_suffix
)

In [ ]:
df

In [ ]:
from collections import Counter

Counter(df["n_neurons"])

In [ ]:
df[df["n_neurons"] == 748]

### split the data into train, val and test

In [ ]:
train, val, test = train_val_test_split(
    df=df, 
    n_neurons_bins=5
)

print(train.shape, val.shape, test.shape)

In [ ]:
train[train["tiff_img_name"] == "SST_11_20.tif"]

train.iloc[34]

# initialise the NeuronPatchDataset object

In [ ]:
train_ds = NeuronPatchDataset(
    df=train,
    ground_truth_config=ground_truth_config,
    pre_processing_config=pre_processing_config,
)

In [ ]:
index = 34
data = train_ds.__getitem__(index=index)

In [ ]:
for key, gt in data.items():
    tifffile.imwrite(
        os.path.join(output_dir,key+f"_{index}_.tiff"),
        data=gt.numpy()
    )

# intialise the dataloader object

In [ ]:
from torch.utils.data import DataLoader

train_dl = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=4, pin_memory=True)